<a href="https://colab.research.google.com/github/rishuatgithub/MLPy/blob/master/Topic_Modelling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modelling using Latent-Dirichlet Allocation

- Blog URL : [Topic Modelling : Latent Dirichlet Allocation, an introduction](https://anotherreeshu.wordpress.com)
- Author   : Rishu Shrivastava

In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### Step 1: Loading Data

As part of this step we will load the data into the dataframe

- Dataset Kernel : https://www.kaggle.com/hengzheng/news-category-classifier-val-acc-0-65

In [30]:
filename = '/content/drive/My Drive/Colab Notebooks/data/news_data/News_Category_Dataset_v2.json'
data = pd.read_json(filename, lines=True)
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


- For the purpose of this blog, we will try to only look at the description of the data and try to generate the topic using Topic Modelling (LDA).

In [0]:
### Selecting only the interested dataset - short_description

df = data['short_description']

In [51]:
df.head()

0    She left her husband. He killed their children...
1                             Of course it has a song.
2    The actor and his longtime girlfriend Anna Ebe...
3    The actor gives Dems an ass-kicking for not fi...
4    The "Dietland" actress said using the bags is ...
Name: short_description, dtype: object

In [52]:
df.shape

(200853,)

### Step 2: Pre-processing data

**a. Applying Count Vectorizer to pre-process the data into vectors.**

In the parameters section of CountVectorizer(), we define the max_df and min_df.

- max_df : Ignore the words that occurs more than 95% of the corpus. 
- min_df : Accept the words in preparation of vocab that occurs in atleast 2 of the documents in the corpus.
- stop_words : Remove the stop words. We can do this in separate steps or in a single step.

In [0]:
df_cv = CountVectorizer(max_df=0.95, min_df=1, stop_words='english')

In [0]:
df_cv_transformed = df_cv.fit_transform(df)

In [55]:
df_cv_transformed

<200853x73729 sparse matrix of type '<class 'numpy.int64'>'
	with 1922096 stored elements in Compressed Sparse Row format>

*Here you can notice that the transformed dataset holds a sparse matrix with a dimension of 200853x73729; where 200853 is the total number of rows and 73729 is the total word corpus.*

### Step 3: Building Latent-Dirichlet Algorithm using scikit-learn

In [0]:
lda_model = LatentDirichletAllocation(n_components=10, batch_size=128, random_state=42)

In [58]:
lda_model.fit(df_cv_transformed)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

### Step 4: Display the Topics

In [75]:
topic_word_dict = {}

for index, topic in enumerate(lda_model.components_):
  topic_id = index
  topic_words_max = [df_cv.get_feature_names()[i] for i in topic.argsort()[-10:]]
  topic_word_dict[topic_id] = topic_words_max
  print(f"Topic ID : {topic_id}; Top 10 Most Words : {topic_words_max}")

Topic ID : 0; Top 10 Most Words : ['movie', 'photos', 'video', 'look', 'film', 'fashion', 'wedding', 'star', 'week', 'new']
Topic ID : 1; Top 10 Most Words : ['government', 'sexual', 'media', 'sleep', 'american', 'women', 'public', 'rights', 'country', 'world']
Topic ID : 2; Top 10 Most Words : ['world', 'killed', 'water', 'week', 'said', 'people', 'police', 'york', 'city', 'new']
Topic ID : 3; Top 10 Most Words : ['election', 'republican', 'clinton', 'obama', 'campaign', 'says', 'donald', 'said', 'president', 'trump']
Topic ID : 4; Top 10 Most Words : ['new', 've', 'little', 'years', 'just', 'time', 'like', 'old', 'day', 'year']
Topic ID : 5; Top 10 Most Words : ['year', 'according', 'court', 'people', 'care', 'new', 'percent', 'women', 'study', 'health']
Topic ID : 6; Top 10 Most Words : ['week', 'day', 'welcome', 'national', 'said', 'united', 'states', 'news', 'house', 'white']
Topic ID : 7; Top 10 Most Words : ['think', 'good', 'really', 'say', 'just', 'people', 'like', 'love', 'do

### Step 5: Attaching Topics to the Dataframe

In [0]:
topic_output = lda_model.transform(df_cv_transformed)

In [63]:
topic_output.argmax(axis=1)

array([4, 0, 0, ..., 0, 1, 2])

In [77]:
topic_word_dict[0]

['movie',
 'photos',
 'video',
 'look',
 'film',
 'fashion',
 'wedding',
 'star',
 'week',
 'new']

In [0]:
data['LDA_Topic_ID'] = topic_output.argmax(axis=1)

data['Topic_word_categories'] = data['LDA_Topic_ID'].apply(lambda id: topic_word_dict[id])

In [83]:
data[['short_description','LDA_Topic_ID','Topic_word_categories']].head()

,short_description,LDA_Topic_ID,Topic_word_categories
0,She left her husband. He killed their children...,4,"[new, ve, little, years, just, time, like, old..."
1,Of course it has a song.,0,"[movie, photos, video, look, film, fashion, we..."
2,The actor and his longtime girlfriend Anna Ebe...,0,"[movie, photos, video, look, film, fashion, we..."
3,The actor gives Dems an ass-kicking for not fi...,3,"[election, republican, clinton, obama, campaig..."
4,"The ""Dietland"" actress said using the bags is ...",7,"[think, good, really, say, just, people, like,..."
